# Демонстрация использования классов `NormalizationCoefficientGenerator` и `OscillogramNormalizer`

Этот ноутбук показывает процесс работы с модулем нормализации:
1.  **`NormalizationCoefficientGenerator`** (`normalization.normalization`): Используется для анализа сырых осциллограмм и генерации CSV-файла с коэффициентами нормализации (`norm.csv`). Этот файл содержит информацию о предполагаемых номинальных значениях, типе подключения (первичка/вторичка) и другие параметры для каждого сигнала в каждой осциллограмме.
2.  **`OscillogramNormalizer`** (`normalization.normalization`): Использует сгенерированный `norm.csv` для приведения данных осциллограмм к нормализованному виду (обычно к относительным единицам).

Также будут продемонстрированы статические методы `NormalizationCoefficientGenerator` для управления файлами коэффициентов.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import sys
import datetime # For custom_datetime in FileOrganizer demo
import csv # For creating sample CSVs

# Настройка путей для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from core.oscillogram import Oscillogram 
from normalization.normalization import NormalizationCoefficientGenerator, OscillogramNormalizer

# Создаем временную директорию для демонстрации
base_sample_dir_norm = "temp_sample_data_normalization"
source_dir_norm_coeffs = os.path.join(base_sample_dir_norm, "source_for_coeff_gen")
output_dir_norm = os.path.join(base_sample_dir_norm, "normalization_output")

if os.path.exists(base_sample_dir_norm):
    shutil.rmtree(base_sample_dir_norm)
os.makedirs(source_dir_norm_coeffs)
os.makedirs(output_dir_norm)

def create_norm_sample_comtrade(path, name, station, analog_channels_data, freq=50.0, samp_rate=1000.0, dat_rows=100):
    cfg_content = f"{station},Device1,1999\n"
    cfg_content += f"{len(analog_channels_data)},{len(analog_channels_data)}A,0D\n"
    for idx, (ch_name, prim_val, sec_val, _) in enumerate(analog_channels_data):
        # Correctly use primary and secondary for transformer ratios, a=1, b=0 for raw data output
        cfg_content += f"{idx+1},{ch_name},A,,V,1.0,0.0,0,-32767,32767,{prim_val},{sec_val},P\n"

    cfg_content += f"{freq}\n1\n{samp_rate},{dat_rows}\n"
    cfg_content += "01/01/2023,00:00:00.000000\n01/01/2023,00:00:00.100000\nASCII\n1.0\n"
    
    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f:
        f.write(cfg_content)

    dat_content = ""
    for i in range(dat_rows):
        dat_content += f"{i+1},{int(i*(1000000/samp_rate))}" # n, timestamp_us
        for _, _, _, values in analog_channels_data:
            dat_content += f",{values[i % len(values)]}" 
        dat_content += "\n"
    with open(os.path.join(path, f"{name}.dat"), "w", encoding="utf-8") as f:
        f.write(dat_content)

create_norm_sample_comtrade(source_dir_norm_coeffs, "osc_norm_voltage", "StationN",
                           [("U_Normal_100V", 10000.0, 100.0, np.sin(np.linspace(0, 2*np.pi*5, 100)) * 100 )],
                           dat_rows=100)

create_norm_sample_comtrade(source_dir_norm_coeffs, "osc_noise_current", "StationN",
                           [("I_Noise", 20.0, 1.0, np.sin(np.linspace(0, 2*np.pi*5, 100)) * 0.01 )],
                           dat_rows=100)

create_norm_sample_comtrade(source_dir_norm_coeffs, "osc_high_primary", "StationP",
                           [("U_HighPrimary", 10000.0, 10.0, np.sin(np.linspace(0, 2*np.pi*5, 100)) * 20 )],
                           dat_rows=100)
                           
signal_distorted = np.sin(np.linspace(0, 2*np.pi*5, 100)) * 50 + np.sin(np.linspace(0, 2*np.pi*15, 100)) * 40
create_norm_sample_comtrade(source_dir_norm_coeffs, "osc_distorted", "StationD",
                           [("U_Distorted", 10000.0, 100.0, signal_distorted)],
                           dat_rows=100)

print(f"Создана директория с примерами для генерации коэффициентов: {source_dir_norm_coeffs}")

## Часть 1: `NormalizationCoefficientGenerator` - Генерация коэффициентов нормализации

In [ ]:
generated_norm_csv_path = os.path.join(source_dir_norm_coeffs, "norm.csv")

coeff_generator = NormalizationCoefficientGenerator(
    osc_path=source_dir_norm_coeffs,
    prev_norm_csv_path="", 
    bus=1 
)
# Для вывода сообщений от NormalizationCoefficientGenerator, если бы у него был параметр is_print_message в __init__:
# coeff_generator.is_print_message = True 

print("\nЗапуск генерации коэффициентов нормализации...")
coeff_generator.normalization() 

if os.path.exists(generated_norm_csv_path):
    print(f"\nФайл коэффициентов '{generated_norm_csv_path}' успешно создан.")
    norm_coeffs_df = pd.read_csv(generated_norm_csv_path)
    print("Содержимое сгенерированного norm.csv:")
    try:
        from IPython.display import display
        display(norm_coeffs_df)
    except ImportError:
        print(norm_coeffs_df)
else:
    print(f"\nОшибка: Файл '{generated_norm_csv_path}' не был создан.")

### Статические методы `NormalizationCoefficientGenerator` (управление CSV)

In [ ]:
norm_additions_content = [
    ["name", "norm", "1Ub_PS", "1Ub_base", "1Ub_h1", "1Ub_hx"],
    ["osc_new_entry", "YES", "s", "100", "95.0", "5.0"]
]
additions_csv_path = os.path.join(output_dir_norm, "norm_additions.csv")
with open(additions_csv_path, "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows(norm_additions_content)

merged_norm_path = os.path.join(output_dir_norm, "norm_merged.csv")
print(f"\nОбъединение {generated_norm_csv_path} и {additions_csv_path}...")
if os.path.exists(generated_norm_csv_path):
    NormalizationCoefficientGenerator.merge_normalization_files_static(
        input_paths_or_folder=[generated_norm_csv_path, additions_csv_path],
        output_csv_path=merged_norm_path,
        is_print_message=True
    )
    if os.path.exists(merged_norm_path):
        print("Содержимое объединенного файла:")
        try: from IPython.display import display; display(pd.read_csv(merged_norm_path))
        except ImportError: print(pd.read_csv(merged_norm_path))
else:
    print(f"Файл {generated_norm_csv_path} не найден, объединение пропущено.")

print("\nДемонстрация correct_df_static (пример вызова, файл для исправления не создается в этом примере):")
corrected_df_path = os.path.join(output_dir_norm, "norm_corrected.csv")
# Для реальной демонстрации, merged_norm_path должен содержать строки с norm='NO' или 'hz'
# NormalizationCoefficientGenerator.correct_df_static(merged_norm_path, corrected_df_path, is_print_message=True)
print("Вызов correct_df_static закомментирован, т.к. требует специально подготовленного файла.")

## Часть 2: `OscillogramNormalizer` - Применение коэффициентов нормализации

In [ ]:
if not os.path.exists(generated_norm_csv_path):
    print(f"Файл коэффициентов {generated_norm_csv_path} не найден. Демонстрация OscillogramNormalizer невозможна.")
else:
    normalizer = OscillogramNormalizer(norm_coef_file_path=generated_norm_csv_path, is_print_message=True)

    osc_to_normalize_path = os.path.join(source_dir_norm_coeffs, "osc_norm_voltage.cfg")

    try:
        osc_obj_to_norm = Oscillogram(osc_to_normalize_path)
        print(f"\nЗагружена осциллограмма для нормализации: {osc_obj_to_norm.file_hash} ({osc_to_normalize_path})")
        
        original_df = osc_obj_to_norm.data_frame.copy()
        
        print("Нормализация данных...")
        # file_identifier должен соответствовать значению в столбце 'name' в norm.csv
        # NormalizationCoefficientGenerator сохраняет file_hash как 'name'.
        normalized_df = normalizer.normalize_bus_signals(
            bus_df=original_df, 
            file_identifier=osc_obj_to_norm.file_hash, # Используем хеш, как и при генерации
            is_print_error=True
        )

        if normalized_df is not None:
            print("\nСравнение данных до и после нормализации (канал 'U_Normal_100V'):")
            # Убедимся, что столбец существует перед доступом
            if 'U_Normal_100V' in original_df.columns and 'U_Normal_100V' in normalized_df.columns:
                comparison_df = pd.DataFrame({
                    'Original U_Normal_100V': original_df['U_Normal_100V'].head(),
                    'Normalized U_Normal_100V': normalized_df['U_Normal_100V'].head()
                })
                try: from IPython.display import display; display(comparison_df)
                except ImportError: print(comparison_df)
                print("Ожидается, что нормализованные значения будут около 0.333, если исходный сигнал ~100В, а _base=100.")
                print("(Это связано с тем, что нормализация напряжений в OscillogramNormalizer делит на 3 * _base)")
            else:
                print("Столбец 'U_Normal_100V' не найден в одном из DataFrame.")
        else:
            print("Нормализация не была применена или вернула None.")

    except Exception as e:
        print(f"Ошибка при демонстрации OscillogramNormalizer: {e}")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_sample_dir_norm):
        shutil.rmtree(base_sample_dir_norm)
        print(f"\nВременная директория {base_sample_dir_norm} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_sample_dir_norm}: {e}")